In [9]:
!pip install -qq openai==0.28
!pip install -qq transformers
!pip install -qq pdfplumber
!pip install -qq python-docx
!pip install -qq transformers datasets accelerate
!pip install -qq huggingface_hub
!pip install -qq pymupdf
!pip install -qq nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 13.4 MB/s eta 0:00:00


In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
import openai
from transformers import pipeline
from huggingface_hub import login
from typing import List
import torch
import gc
import fitz  # PyMuPDF
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: using Google CoLab


In [13]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_ACCESS_TOKEN')

In [14]:
# Function to generate text using OpenAI API
def openai_generate_text(prompt, api_key, model="gpt-4o-mini", max_tokens=256, temperature=0.5):
    openai.api_key = api_key

    try:
        # Call OpenAI's GPT model to generate text
        response = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )

        # Extract generated text from response
        generated_text = response['choices'][0]['message']['content'].strip()
        return generated_text

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

## Extract Text from Document

In [15]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path, pipe):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    # return text
    results = pipe(text, truncation=True, max_length=512)
    print(results)
    return text

# Extract Experience Only

In [16]:
def extract_experience_only(raw_experience):
    if isinstance(raw_experience, list):
        raw_experience = '\n'.join([content for _, content in raw_experience])

    experience_sections = []
    capture_experience = False

    end_section_keywords = ["Certifications", "Certificate", "Certified", "Work Experience",
                            "Referees", "Skills", "Awards", "Publications", "Courses", "Conferences", "Memberships"]
    end_section_keywords = [keyword.lower() for keyword in end_section_keywords]
    experience_keywords = ["Professional Experience", "Experience", "Work History", "Work Experience"]
    experience_keywords = [keyword.lower() for keyword in experience_keywords]

    for line in raw_experience.split('\n'):
        line_clean = line.strip()
        if not line_clean:
            continue

        if any(keyword in line.lower() for keyword in experience_keywords):
            capture_experience = True
            continue

        if any(keyword in line.lower() for keyword in end_section_keywords):
            capture_experience = False

        if capture_experience:
            experience_sections.append(line_clean)

    return experience_sections


In [17]:
def get_job_titles(resume, api_key):
    jobs = openai_generate_text(f"Extract the job titles, period they were held and location from this resume: '{resume}'", api_key)
    return jobs

In [18]:
# Initialize the question-answering pipeline
def get_candidate_name(resume_text):
    qa_pipe = pipeline("question-answering", model="Kiet/autotrain-resume_parser-1159242747")
    question = "What is the candidate's name?"
    result = qa_pipe(question=question, context=resume_text)
    answer = result['answer']

    return answer

## Prepare output document

In [71]:
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.text import WD_LINE_SPACING
from docx.shared import Cm
import re

In [67]:
# import re
# from docx import Document
# from docx.shared import Cm, Pt
# from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

# def create_resume_doc(jobs, refined, name, education, profile, file_name="Refined_Resume.docx"):
#     # Create a new Document
#     doc = Document()

#     # Set the document to A4 size
#     section = doc.sections[0]
#     section.page_height = Cm(29.7)
#     section.page_width = Cm(21.0)

#     # Set margins for A4
#     section.left_margin = Cm(2)
#     section.right_margin = Cm(2)
#     section.top_margin = Cm(2)
#     section.bottom_margin = Cm(2)


#     # Title of the Resume
#     title = doc.add_heading("RESUME", level=1)
#     title.paragraph_format.space_after = Pt(0)
#     title.paragraph_format.space_before = Pt(0)
#     title.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
#     name_paragraph = doc.add_paragraph(name)
#     name_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

#     # Profile Section
#     if profile:
#         doc.add_paragraph("Profile", style='Heading 2')
#         achievement_lines = profile.split("\n")
#         for line in achievement_lines:
#             if line.strip():
#                 paragraph = doc.add_paragraph(line.strip())
#                 paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY

#     # Add Experience Section to document
#     doc.add_paragraph("Tailored Experience", style='Heading 2')
#     experience_lines = refined.split("\n")
#     for line in experience_lines:
#         cleaned_line = line.strip()

#         if cleaned_line.lower() == "experience" or not cleaned_line:
#             continue

#         cleaned_line = re.sub(r'[^\w\s\,–-]', '', cleaned_line)

#         # Check if the line is a job title
#         paragraph = None
#         if any(job in cleaned_line for job in jobs):
#             paragraph = doc.add_paragraph(cleaned_line)
#             paragraph.paragraph_format.space_after = Pt(0)

#         else:
#             if len(cleaned_line) > 3:
#                 paragraph = doc.add_paragraph(cleaned_line, style='List Bullet')

#         if paragraph:
#             paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY

#     # Add Education section to the document
#     doc.add_paragraph("Education", style='Heading 2')
#     education_lines = education.split("\n")
#     for line in education_lines:
#         if line.strip():
#             cleaned_line = line.lstrip('- ').strip()
#             paragraph = doc.add_paragraph(cleaned_line)
#             paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY

#     # Save the document
#     doc.save(file_name)
#     print(f"Document saved as {file_name}")


In [72]:
def add_experience_section(doc, refined, jobs):
    doc.add_paragraph("Tailored Experience", style='Heading 2')

    if refined:
        experience_lines = refined.split("\n")

        first_line = experience_lines[0].strip().lower()
        if "experience" in first_line or "heres a refined experience section" in first_line:
            experience_lines = experience_lines[1:]

        for line in experience_lines:
            cleaned_line = line.strip()
            paragraph = None

            if cleaned_line.lower() == "experience" or len(cleaned_line) < 2:
                continue

            cleaned_line = re.sub(r'[^\w\s,–-•]', '', cleaned_line)

            if any(job.lower() in cleaned_line.lower() for job in jobs):
                paragraph = doc.add_paragraph()
                run = paragraph.add_run(cleaned_line)
                # run.bold = True

            else:
                if len(cleaned_line) > 3:
                    paragraph = doc.add_paragraph(cleaned_line, style='List Bullet')
                    run = paragraph.add_run()
                    run.italic = True

            if paragraph:
                paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
                paragraph.paragraph_format.line_spacing_rule = WD_LINE_SPACING.SINGLE

In [73]:
def add_education_section(doc, education):
    if education:
        education_lines = education.split("\n")

        current_entry = []

        for line in education_lines:
            cleaned_line = re.sub(r'[^\w\s,–-]', '', line).strip()

            if not cleaned_line:
                continue

            if "MSc" in cleaned_line or "BSc" in cleaned_line:
                if current_entry:
                    add_education_paragraph(doc, current_entry)
                    current_entry = []

            current_entry.append(cleaned_line)

        if current_entry:
            add_education_paragraph(doc, current_entry)

def add_education_paragraph(doc, entry_lines):
    paragraph = doc.add_paragraph(style=None)
    for i, line in enumerate(entry_lines):
        run = paragraph.add_run(line)
        # run.italic = True

        if i < len(entry_lines) - 1:
            run.add_break()

    paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
    paragraph.paragraph_format.line_spacing_rule = WD_LINE_SPACING.SINGLE
    paragraph.paragraph_format.space_after = Pt(12)


In [74]:
def create_resume_doc(jobs, refined, name, education, profile, file_name="Refined_Resume.docx"):
    # Create a new Document
    doc = Document()

    # Set the document to A4 size
    section = doc.sections[0]
    section.page_height = Cm(29.7)
    section.page_width = Cm(21.0)
    section.font_size = Pt(12)
    section.font_name = 'Times New Roman'

    # Set margins for A4
    section.left_margin = Cm(2)
    section.right_margin = Cm(2)
    section.top_margin = Cm(2)
    section.bottom_margin = Cm(2)


    # Title and Name of the Resume
    title = doc.add_heading("RESUME", level=1)
    title.paragraph_format.space_after = Pt(0)
    title.paragraph_format.space_before = Pt(0)
    title.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    name = doc.add_paragraph(name)
    name.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    name.paragraph_format.space_after = Pt(0)
    name.paragraph_format.space_before = Pt(0)

    # Add Profile Section
    if profile:
        doc.add_paragraph("Profile", style='Heading 2')
        achievement_lines = profile.split("\n")
        for line in achievement_lines:
            if line.strip():
                paragraph = doc.add_paragraph(line.strip())
                paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY

    # Add Experience Section to document
    add_experience_section(doc, refined, jobs)

    # Add Education section to the document
    doc.add_paragraph("Education", style='Heading 2')
    add_education_section(doc, education)

    # Save the document
    doc.save(file_name)
    print(f"Document saved as {file_name}")

In [75]:
def main():
    api_key = userdata.get('OPENAI_API_KEY')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Extract text from documents
    extract_pipeline = pipeline("text-classification", model="nikunjbjj/jd-resume-model", device=device)
    resume_pdf_path = "Pauline Korukundo Resume 2024.pdf"
    resume = extract_text_from_pdf(resume_pdf_path, extract_pipeline)
    job_descr_pdf_path = "software_eng_1_yr.pdf"
    job_descr = extract_text_from_pdf(job_descr_pdf_path, extract_pipeline)

    # Generate a profile summary
    profile_prompt = (
        f"Generate a brief, two-sentence professional summary highlighting key skills and experiences from the resume. "
        f"Focus on achievements relevant to a software engineering role. Avoid including a title: '{resume}'"
    )
    profile = openai_generate_text(profile_prompt, api_key)

    # Extract and refine experience to match job description
    get_experience = extract_experience_only(resume)
    refined_exp_prompt = (
        f"Refine the experience section of this resume to match the requirements in the job description. "
        f"Summarize each position with a maximum of 3 bullet points, focusing on achievements, skills, or responsibilities most relevant to the job: '{job_descr}'. "
        f"Here is the experience to be refined: '{get_experience}'"
    )
    refined_exp = openai_generate_text(refined_exp_prompt, api_key)

    # Extract education details
    education_prompt = (
        f"Extract the education details from this resume. For each institution, list the degree, institution name, dates attended, and completion status. "
        f"Format each entry as a bullet point, with the completion status appearing at the end of each entry: '{resume}'"
    )
    education = openai_generate_text(education_prompt, api_key)

    # Extract job titles for formatting
    jobs = get_job_titles(resume, api_key)

    # Extract candidate name
    name = get_candidate_name(resume)

    # Generate the new resume document
    create_resume_doc(jobs, refined_exp, name, education, profile)

if __name__ == "__main__":
    main()

[{'label': 'POS', 'score': 0.4816814064979553}]
[{'label': 'POS', 'score': 0.5531230568885803}]


Some weights of the model checkpoint at Kiet/autotrain-resume_parser-1159242747 were not used when initializing LongformerForQuestionAnswering: ['longformer.embeddings.position_ids']
- This IS expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Document saved as Refined_Resume.docx
